In [1]:
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.stem.porter import PorterStemmer
from nltk.corpus import stopwords
import nltk
import re
from sklearn.naive_bayes import MultinomialNB, BernoulliNB, GaussianNB
import pandas as pd
import string
from sklearn import feature_extraction, linear_model, model_selection, preprocessing
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

In [2]:
# Opening the file
f = open("amazon_cells_labelled.txt", "r")

data =[]
# Converting it to pandas dataframe
for line in f:
    review = line[:len(line) - 2]
    sentiment = line[len(line)-2]
    row = [review, sentiment]
    data.append(row)

df = pd.DataFrame(data, columns = ['reviews', 'sentiment'])

In [3]:
# Opening the file
f = open("yelp_labelled.txt", "r")

data =[]
# Converting it to pandas dataframe
for line in f:
    review = line[:len(line) - 2]
    sentiment = line[len(line)-2]
    row = [review, sentiment]
    data.append(row)

df1 = pd.DataFrame(data, columns = ['reviews', 'sentiment'])

In [4]:
# Opening the file
f = open("imdb_labelled.txt", "r")

data =[]
# Converting it to pandas dataframe
for line in f:
    review = line[:len(line) - 2]
    sentiment = line[len(line)-2]
    row = [review, sentiment]
    data.append(row)

df2 = pd.DataFrame(data, columns = ['reviews', 'sentiment'])

In [6]:
len(df)

1000

In [7]:
df = df.append(df1).append(df2)

In [8]:
len(df)

3000

In [11]:
import pandas as pd
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

#removing the stop words from the corpus

stop = stopwords.words('english') 


df['reviews'] = df.reviews.str.replace("[^\w\s]", "").str.lower()
df['reviews'] = df['reviews'].apply(lambda x: ' '.join([item for item in x.split() if item not in stop]))

<ipython-input-11-ab282e7c83b5>:10: FutureWarning: The default value of regex will change from True to False in a future version.
  df['reviews'] = df.reviews.str.replace("[^\w\s]", "").str.lower()


In [12]:
df.head()

,reviews,sentiment
0,way plug us unless go converter,0
1,good case excellent value,1
2,great jawbone,1
3,tied charger conversations lasting 45 minutesm...,0
4,mic great,1


In [13]:
#function to remove numeric characters

def remove_numric_character(x):
    x = re.sub(r'\b[0-9]+\b\s*', '', x)
    return x   

#function to remove punctuations
def remove_punctuations(text):
    for punctuation in string.punctuation:
        text = text.replace(punctuation, '')
    return text


#function to clean the string
def clean_string(s):
    tokens = s.strip().split()
    clean_tokens = [t for t in tokens if re.match(r'[^\W\d]*$', t)]
    clean_s = ' '.join(clean_tokens)
    return clean_s


In [14]:
#mapping the above functions

df["reviews"] = df['reviews'].apply(remove_punctuations)
df["reviews"] = df['reviews'].apply(remove_numric_character)
df["reviews"] = df['reviews'].apply(clean_string)

In [15]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3000 entries, 0 to 999
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   reviews    3000 non-null   object
 1   sentiment  3000 non-null   object
dtypes: object(2)
memory usage: 70.3+ KB


In [16]:
#train test split and creating pipeline

x_train,x_test,y_train,y_test = train_test_split(df["reviews"], df.sentiment, test_size=0.33, random_state=20)
pipe_lr = Pipeline([('tfidf', TfidfVectorizer()), ('clf', LogisticRegression(random_state=42))])

In [17]:
#parameters for logistic regression

param_range_fl = [0.1, 1, 10,  100]
grid_params_lr = [{'clf__penalty': ['l1', 'l2'],
        'clf__C': param_range_fl,
        'clf__solver': ['newton-cg','liblinear']}] 

In [28]:
#Logistic regression grid CV

LR = GridSearchCV(estimator=pipe_lr,
            param_grid=grid_params_lr,
            scoring='roc_auc',
            cv=10 , verbose = 3 ) 

In [29]:
LR.fit(x_train, y_train)

Fitting 10 folds for each of 16 candidates, totalling 160 fits
[CV 1/10] END clf__C=0.1, clf__penalty=l1, clf__solver=newton-cg;, score=nan total time=   0.0s
[CV 2/10] END clf__C=0.1, clf__penalty=l1, clf__solver=newton-cg;, score=nan total time=   0.0s
[CV 3/10] END clf__C=0.1, clf__penalty=l1, clf__solver=newton-cg;, score=nan total time=   0.0s
[CV 4/10] END clf__C=0.1, clf__penalty=l1, clf__solver=newton-cg;, score=nan total time=   0.0s
[CV 5/10] END clf__C=0.1, clf__penalty=l1, clf__solver=newton-cg;, score=nan total time=   0.0s
[CV 6/10] END clf__C=0.1, clf__penalty=l1, clf__solver=newton-cg;, score=nan total time=   0.0s
[CV 7/10] END clf__C=0.1, clf__penalty=l1, clf__solver=newton-cg;, score=nan total time=   0.0s
[CV 8/10] END clf__C=0.1, clf__penalty=l1, clf__solver=newton-cg;, score=nan total time=   0.0s
[CV 9/10] END clf__C=0.1, clf__penalty=l1, clf__solver=newton-cg;, score=nan total time=   0.0s
[CV 10/10] END clf__C=0.1, clf__penalty=l1, clf__solver=newton-cg;, score

/Users/sharanbasavasumbad/python-env/env/lib/python3.9/site-packages/sklearn/svm/_base.py:1206: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


[CV 1/10] END clf__C=10, clf__penalty=l1, clf__solver=liblinear;, score=0.818 total time=   2.3s


/Users/sharanbasavasumbad/python-env/env/lib/python3.9/site-packages/sklearn/svm/_base.py:1206: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


[CV 2/10] END clf__C=10, clf__penalty=l1, clf__solver=liblinear;, score=0.865 total time=   2.4s
[CV 3/10] END clf__C=10, clf__penalty=l1, clf__solver=liblinear;, score=0.788 total time=   0.0s


...

[CV 8/10] END clf__C=10, clf__penalty=l1, clf__solver=liblinear;, score=0.827 total time=   2.2s
[CV 9/10] END clf__C=10, clf__penalty=l1, clf__solver=liblinear;, score=0.884 total time=   0.0s


/Users/sharanbasavasumbad/python-env/env/lib/python3.9/site-packages/sklearn/svm/_base.py:1206: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


[CV 10/10] END clf__C=10, clf__penalty=l1, clf__solver=liblinear;, score=0.888 total time=   2.4s
[CV 1/10] END clf__C=10, clf__penalty=l2, clf__solver=newton-cg;, score=0.861 total time=   0.0s
[CV 2/10] END clf__C=10, clf__penalty=l2, clf__solver=newton-cg;, score=0.885 total time=   0.0s
[CV 3/10] END clf__C=10, clf__penalty=l2, clf__solver=newton-cg;, score=0.831 total time=   0.0s
[CV 4/10] END clf__C=10, clf__penalty=l2, clf__solver=newton-cg;, score=0.873 total time=   0.0s
[CV 5/10] END clf__C=10, clf__penalty=l2, clf__solver=newton-cg;, score=0.876 total time=   0.0s
[CV 6/10] END clf__C=10, clf__penalty=l2, clf__solver=newton-cg;, score=0.871 total time=   0.0s
[CV 7/10] END clf__C=10, clf__penalty=l2, clf__solver=newton-cg;, score=0.875 total time=   0.0s
[CV 8/10] END clf__C=10, clf__penalty=l2, clf__solver=newton-cg;, score=0.855 total time=   0.0s
[CV 9/10] END clf__C=10, clf__penalty=l2, clf__solver=newton-cg;, score=0.897 total time=   0.0s
[CV 10/10] END clf__C=10, clf

/Users/sharanbasavasumbad/python-env/env/lib/python3.9/site-packages/sklearn/model_selection/_validation.py:372: FitFailedWarning: 
40 fits failed out of a total of 160.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
40 fits failed with the following error:
Traceback (most recent call last):
  File "/Users/sharanbasavasumbad/python-env/env/lib/python3.9/site-packages/sklearn/model_selection/_validation.py", line 680, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/Users/sharanbasavasumbad/python-env/env/lib/python3.9/site-packages/sklearn/pipeline.py", line 394, in fit
    self._final_estimator.fit(Xt, y, **fit_params_last_step)
  File "/Users/sharanbasavasumbad/python-env/env/lib/python3.9/site-packages/skl

GridSearchCV(cv=10,
             estimator=Pipeline(steps=[('tfidf', TfidfVectorizer()),
                                       ('clf',
                                        LogisticRegression(random_state=42))]),
             param_grid=[{'clf__C': [0.1, 1, 10, 100],
                          'clf__penalty': ['l1', 'l2'],
                          'clf__solver': ['newton-cg', 'liblinear']}],
             scoring='roc_auc', verbose=3)

In [30]:
print("Best: %f using %s" % (LR.best_score_,LR.best_params_))

Best: 0.873439 using {'clf__C': 1, 'clf__penalty': 'l2', 'clf__solver': 'newton-cg'}


In [31]:
print("Logistic Regression best grid score: " + str(LR.best_score_))
print("Logistic Regression grid test score: " + str(LR.score(x_test, y_test)))

LR_best_params = LR.best_params_
print("Logistic Regression best params: " + str(LR_best_params))

Logistic Regression best grid score: 0.8734389767509754
Logistic Regression grid test score: 0.8798686543209371
Logistic Regression best params: {'clf__C': 1, 'clf__penalty': 'l2', 'clf__solver': 'newton-cg'}


In [33]:
!jupyter-nbconvert --to PDFviaHTML Assignment9_sharanbasav.ipynb

[NbConvertApp] Converting notebook Assignment9_sharanbasav.ipynb to PDFviaHTML
[NbConvertApp] Writing 175944 bytes to Assignment9_sharanbasav.pdf
